<a href="https://colab.research.google.com/github/Oichii/CosmeticsScan/blob/master/SNR_Projekt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Comparison of two deep convolutional neural networks architectures

In [0]:
import keras
from keras.applications import ResNet50, VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, MaxPooling2D, Activation, Flatten, Dense, ZeroPadding2D, Dropout
from keras.models import Model
from keras.optimizers import Adam, SGD
from sklearn.metrics import classification_report
from keras import models, optimizers
from keras.callbacks import CSVLogger, ModelCheckpoint

Using TensorFlow backend.


Mounting drive containing datasets

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
path_drive = '/content/gdrive/My Drive'

import os
os.chdir('/content/gdrive/My Drive/snr_unpacked/')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


VGG network

In [0]:
def VGG(img_height, img_width):
  model = models.Sequential()
  base_model = VGG16(include_top=False, input_shape=(img_height, img_width, 3), weights='imagenet')
  base_model.summary()
  for layer in base_model.layers:
      layer.trainable = False
  model.add(base_model)
  model.add(Flatten())
  model.add(Dense(units=1024,activation="relu"))
  model.add(Dense(units=512,activation="relu"))
  model.add(Dense(units=2, activation='softmax'))

  for layer in model.layers[:8]:
      layer.trainable=True

  return model


LeNet network

In [0]:
def LeNet(img_height, img_width):

    model = models.Sequential()

    model.add(Conv2D(32, (3, 3), input_shape=(img_height, img_width, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(units=512,activation="relu"))
    model.add(Dense(units=256,activation="relu"))

    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    return model


In [0]:
import tensorflow as tf 
def get_session(gpu_fraction=0.3):
  num_threads = os.environ.get('OMP_NUM_THREADS')
  gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=gpu_fraction)

  if num_threads:
      return tf.Session(config=tf.ConfigProto(
          gpu_options=gpu_options, intra_op_parallelism_threads=num_threads))
  else:
      return tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 8} )
sess = tf.Session(config=config)
keras.backend.set_session(get_session())

Preparing data

In [0]:
image_shape = (776, 200)
img_height = 200
img_width = 776
model = LeNet(img_height, img_width)

In [0]:
train_datagen = ImageDataGenerator(
    rotation_range=4,
    # horizontal_flip=True,
    vertical_flip=True,
    rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        'train',
        target_size=(img_height, img_width),
        batch_size=4)

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
        'test',
        target_size=(img_height, img_width),
        batch_size=4)

Found 646 images belonging to 2 classes.
Found 162 images belonging to 2 classes.


In [0]:
def noise_generator(batches):
    """Take as input a Keras ImageGen (Iterator) and generate random
    crops from the image batches generated by the original iterator.
    """
    import numpy as np
    import cv2
    while True:
        batch_x, batch_y = next(batches)
        batch_crops = np.zeros(batch_x.shape)
        for i in range(batch_x.shape[0]):
          if np.random.randint(0,1):
            noise = np.random.normal(0, np.random.randint(0, 50)/ 255.0, batch_x[i].shape)
            batch_crops[i] = batch_x[i]+noise
          else:
            batch_crops[i] = cv2.GaussianBlur(batch_x[i],(3,3),0)
        yield (batch_crops, batch_y)

# Training

In [0]:
model.compile(optimizer=Adam(0.0001), loss='categorical_crossentropy', metrics=['accuracy'])  # precision, recall and f1 are misleading for batch processing
model.summary()
save_dir = '/content/gdrive/My Drive/SNR/modele'
save_every = 1
checkpointer = ModelCheckpoint(os.path.join(save_dir, 'model_VGG{epoch:03d}.hdf5'), verbose=1, save_weights_only=False, period=save_every)

csv_logger = CSVLogger(os.path.join(save_dir, 'log.csv'), append=True, separator=',')

step_size_train=train_generator.n // train_generator.batch_size

step_size_test = test_generator.n // train_generator.batch_size

history = model.fit_generator(generator=noise_generator(train_generator),
                   steps_per_epoch=step_size_train,
                validation_data=test_generator,
                validation_steps=step_size_test,
                   epochs=20, callbacks=[checkpointer, csv_logger])



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 198, 774, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 198, 774, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 99, 387, 32)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 97, 385, 32)       9248      
_________________________________________________________________
activation_2 (Activation)    (None, 97, 385, 32)       0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 48, 192, 32)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 46, 190, 64)    

ValueError: ignored

In [0]:
from keras.models import load_model
model = load_model('/content/gdrive/My Drive/SNR/modele/model_VGG007.hdf5', compile=False)

y_pred = model.predict_generator(test_generator, verbose=1)

# y_pred_bool = np.argmax(y_pred, axis=1)

# Plots

Classification report and confusion matrix

In [0]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, precision_recall_curve
print(test_generator.classes)
print(y_pred)
y_class=[]
for y in y_pred:
  if y[0]>y[1]:
    y_class.append(0)
  else:
    y_class.append(1)
  print(y, y_class[-1])

print(classification_report(test_generator.classes, np.around(y_class)))
print(confusion_matrix(test_generator.classes, np.around(y_class)))


Accuracy

In [0]:
import matplotlib.pyplot as plt
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.grid()
plt.show()

Loss

In [0]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.grid()
plt.legend(['train', 'test'], loc='upper left')
plt.show()

ROC and precision-recall

In [0]:
y_class=[]
for y in y_pred:
  if y[0]>y[1]:
    y_class.append(y[0])
  else:
    y_class.append(y[1])
ns_fpr, ns_tpr, _ = roc_curve(test_generator.classes, y_class)
plt.plot(ns_fpr, ns_tpr, linestyle='--')
plt.grid()
plt.show()
precision, recall, thresholds = precision_recall_curve(test_generator.classes, y_class)
plt.plot(recall, precision, marker='.')
plt.grid()
plt.show()
